In [1]:
# 서프라이즈 설치   
# !pip install surprise

In [7]:
from surprise import SVD
from surprise.dataset import DatasetAutoFolds
from surprise.dataset import Reader
import pymysql
import pandas as pd
from surprise import accuracy

In [3]:
# ratings 가져오기 
# 1. connect
conn = pymysql.connect(host="3.38.186.130", port=3306, db="finalteam3", user="kdigital3", password="mysql", charset="utf8")

# 2. get command object
cursor = conn.cursor()

# 3. execute command
sql = """select member_id, movie_id, rating
        from rating"""
cursor.execute(sql)
    
    # break

conn.commit() # confirm previous execution

# 4. close resource
cursor.close()
conn.close()

movie = []
list = []
rows = cursor.fetchall()
ratings = pd.DataFrame(rows, columns=['member_id','movie_id','rating'])

In [4]:
# movie데이터 가져오기
# 1. connect
conn = pymysql.connect(host="127.0.0.1", port=3306, db="finalteam3", user="kdigital", password="mysql", charset="utf8")

# 2. get command object
cursor = conn.cursor()

# 3. execute command
sql = """select title, movie_id, vote_average
        from movie"""
cursor.execute(sql)
    
    # break

conn.commit() # confirm previous execution

# 4. close resource
cursor.close()
conn.close()

rows = cursor.fetchall()
movies = pd.DataFrame(rows, columns=['title','movie_id','vote_average'])


In [5]:
# reader = Reader(line_format='user item rating', sep=',',
#                rating_scale=(1, 10))

# # DatasetAutoFolds 클래스를 사용해서 개별적으로 생성
# # index와 header가 없는 상태로 재생성했던 ratings_surprise.csv파일에 기반
# data_folds = DatasetAutoFolds(ratings_file='rating_surprise.csv',
#                              reader=reader)
from surprise import Dataset,reader
from surprise.model_selection import train_test_split
import pandas as pd

# 내장 데이터인 무비렌즈 데이터 로드하고 학습/테스트 데이터로 분리
data = Dataset.load_builtin('ml-100k')
# df = pd.read_csv('../data-files/ratings.csv')
# df = df[['userId','movieId','rating']]
# reader = Reader(rating_scale=(1,10))
# data = Dataset.load_from_df(df[['userId','movieId','rating']],reader)
train = data.build_full_trainset()
test = train.build_testset()
# train, test = train_test_split(data, test_size=0.01, random_state=42, shuffle=True)


In [50]:
from surprise.model_selection import GridSearchCV
from surprise import accuracy

param_grid = {'n_epochs':[20,30,40,50,60], 'n_factors':[10,20,30,40,50,60,100,200],'lr_all':[0.5,0.05],'reg_all':[0.06,0.04]}
gs = GridSearchCV(SVD, param_grid, measures=['RMSE','MAE'], cv=3,n_jobs=-1)
gs.fit(data)
print(gs.best_params['rmse'])
print(gs.best_score['rmse'])
print()
print(gs.best_params['mae'])
print(gs.best_score['mae'])

{'n_epochs': 20, 'n_factors': 200, 'lr_all': 0.05, 'reg_all': 0.06}
0.9352130399849222

{'n_epochs': 20, 'n_factors': 200, 'lr_all': 0.05, 'reg_all': 0.06}
0.7391771256526362


In [28]:
algo = SVD(n_factors=200, n_epochs=20, lr_all=0.06, reg_all=0.01,random_state=42)
algo.fit(train)
prediction = algo.test(test)
accuracy.rmse(prediction)

RMSE: 0.0840


0.08397072665685956

In [22]:
ratings['rating'] = ratings['rating']/2

In [10]:
def get_unseen_surprise(ratings, movies, member_id):
    # 특정 유저가 본 movie id들을 리스트로 할당
    seen_movies = ratings[ratings['member_id']==member_id]['movie_id'].tolist()
    # 모든 영화들의 movie id들 리스트로 할당
    total_movies = movies['movie_id'].tolist()
    
    # 모든 영화들의 movie id들 중 특정 유저가 본 movie id를 제외한 나머지 추출
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print(f'특정 {member_id} 유저가 본 영화 수: {len(seen_movies)}\n추천한 영화 개수: {len(unseen_movies)}\n전체 영화수: {len(total_movies)}')
    
    return unseen_movies

In [24]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict()를 이용해 특정 userId의 평점이 없는 영화들에 대해 평점 예측
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

    # predictions는 Prediction()으로 하나의 객체로 되어있기 때문에 예측평점(est값)을 기준으로 정렬해야함
    # est값을 반환하는 함수부터 정의. 이것을 이용해 리스트를 정렬하는 sort()인자의 key값에 넣어주자!
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est함수로 리스트를 정렬하는 sort함수의 key인자에 넣어주자
    # 리스트 sort는 디폴트값이 inplace=True인 것처럼 정렬되어 나온다. reverse=True가 내림차순
    predictions.sort(key=sortkey_est, reverse=True)
    # 상위 n개의 예측값들만 할당
    top_predictions = predictions[:top_n]
    
    # top_predictions에서 movie id, rating, movie title 각 뽑아내기
    top_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_movie_ratings = [pred.est*2 for pred in top_predictions]
    top_movie_titles = movies[movies.movie_id.isin(top_movie_ids)]['title']
    # 위 3가지를 튜플로 담기
    # zip함수를 사용해서 각 자료구조(여기선 리스트)의 똑같은 위치에있는 값들을 mapping
    # zip함수는 참고로 여러개의 문자열의 똑같은 위치들끼리 mapping도 가능!
    top_movie_preds = [(ids, rating, title) for ids, rating, title in zip(top_movie_ids, top_movie_ratings, top_movie_titles)]
    
    return top_movie_preds

In [25]:
### 위에서 정의한 함수를 사용해 특정 유저의 추천 영화들 출력해보기
unseen_lst = get_unseen_surprise(ratings, movies, 'admin1234')
top_movies_preds = recomm_movie_by_surprise(algo, 'admin1234', unseen_lst,
                                           top_n=10)
print()
print('#'*8,'Top-10 추천영화 리스트','#'*8)

# top_movies_preds가 여러가지의 튜플을 담고 있는 리스트이기 때문에 반복문 수행
for top_movie in top_movies_preds:
    print('* 추천 영화 이름: ', top_movie[2])
    print('* 해당 영화의 예측평점: ', top_movie[1])
    print()

특정 admin1234 유저가 본 영화 수: 30
추천한 영화 개수: 527513
전체 영화수: 527543

######## Top-10 추천영화 리스트 ########
* 추천 영화 이름:  글래디에이터
* 해당 영화의 예측평점:  8.889786142432618

* 추천 영화 이름:  스타 트렉 6: 미지의 세계
* 해당 영화의 예측평점:  8.770587179352905

* 추천 영화 이름:  분노의 폭발
* 해당 영화의 예측평점:  8.77027356792394

* 추천 영화 이름:  배트맨 비긴즈
* 해당 영화의 예측평점:  8.73370609918668

* 추천 영화 이름:  Halbe Treppe
* 해당 영화의 예측평점:  8.693235090449917

* 추천 영화 이름:  밀리언 달러 호텔
* 해당 영화의 예측평점:  8.655319444719911

* 추천 영화 이름:  백설공주와 일곱 난쟁이
* 해당 영화의 예측평점:  8.642709679314391

* 추천 영화 이름:  몬순 웨딩
* 해당 영화의 예측평점:  8.61732854949178

* 추천 영화 이름:  광란의 사랑
* 해당 영화의 예측평점:  8.539531217825825

* 추천 영화 이름:  매트릭스
* 해당 영화의 예측평점:  8.53402380655392



In [26]:
unseen_lst = get_unseen_surprise(ratings, movies, 'iamuser')
top_movies_preds = recomm_movie_by_surprise(algo, 'iamuser', unseen_lst,
                                           top_n=10)
print()
print('#'*8,'Top-10 추천영화 리스트','#'*8)

# top_movies_preds가 여러가지의 튜플을 담고 있는 리스트이기 때문에 반복문 수행
for top_movie in top_movies_preds:
    print('* 추천 영화 이름: ', top_movie[2])
    print('* 해당 영화의 예측평점: ', top_movie[1])
    print()

특정 iamuser 유저가 본 영화 수: 10
추천한 영화 개수: 527533
전체 영화수: 527543

######## Top-10 추천영화 리스트 ########
* 추천 영화 이름:  니모를 찾아서
* 해당 영화의 예측평점:  8.889786142432618

* 추천 영화 이름:  그녀에게
* 해당 영화의 예측평점:  8.850485751069364

* 추천 영화 이름:  귀여운 여인
* 해당 영화의 예측평점:  8.830402859176386

* 추천 영화 이름:  프레데터 2
* 해당 영화의 예측평점:  8.770587179352905

* 추천 영화 이름:  스타 트렉 6: 미지의 세계
* 해당 영화의 예측평점:  8.77027356792394

* 추천 영화 이름:  배트맨 비긴즈
* 해당 영화의 예측평점:  8.757189528006064

* 추천 영화 이름:  밀리언 달러 호텔
* 해당 영화의 예측평점:  8.737337014964957

* 추천 영화 이름:  백설공주와 일곱 난쟁이
* 해당 영화의 예측평점:  8.73370609918668

* 추천 영화 이름:  광란의 사랑
* 해당 영화의 예측평점:  8.693235090449917

* 추천 영화 이름:  매트릭스
* 해당 영화의 예측평점:  8.655319444719911

